In [1]:
########### Python 3.2 #############
import requests
import json
import pandas as pd
import datetime as dt

############

In [2]:
!pip install ecmwf-api-client

In [5]:
url = "https://api.energyville.be/smarthordc/v1/"
source = 'Weather.Forecast.Rebase'
params = {
    'start': '2020-08-01T00:00:00Z',
    'end': '2020-08-31T00:00:00Z',
    'keys': ['Temperature', 'CloudCover', 'SolarDownwardRadiation', 'SurfaceDownSolarDiffuseRadiation', 'SurfaceDownSolarDirectRadiation', 'WindSpeed', 'RelativeHumidity']
}
body = {
    "Location": ["EnergyVille 1, BE"]
}
headers = {
    'Ocp-Apim-Subscription-Key': '1e3353b6342142cd96b61451416bd9cb',
}
response = requests.post(f'{url}/api/sources/{source}/timeseries?', headers=headers, params=params, json=body)
df = pd.DataFrame(json.loads(response.text))
df.head()

""


In [4]:
#Only use the forecast at 11 o clock
# Convert ForecastGeneratedTimestamp to datetime if it's not already in datetime format
df['ForecastGeneratedTimestamp'] = pd.to_datetime(df['ForecastGeneratedTimestamp'])
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Round the ForecastGeneratedTimestamp to the nearest hour
df['ForecastGeneratedTimestamp'] = df['ForecastGeneratedTimestamp'].dt.round('H')

# Select rows with '11:00:00' in RoundedForecastGeneratedTimestamp
DA_forecast_datetimes = pd.to_datetime(df['Timestamp'].dt.date - pd.Timedelta(days=1), utc = True) + pd.Timedelta(hours=11)
DA_forecasts = df[df['ForecastGeneratedTimestamp'] == DA_forecast_datetimes]

# Drop unnecessary columns
#closest_forecasts = closest_forecasts.drop('RoundedForecastGeneratedTimestamp', axis=1)

# Display or use the resulting DataFrame (closest_forecasts)
DA_forecasts.head()


KeyError: 'ForecastGeneratedTimestamp'

In [ ]:
DA_forecasts = DA_forecasts.pivot(columns='Key', index='Timestamp', values='Value')

In [ ]:
import pickle
with open('Rebase.pickle', 'wb') as f:
    pickle.dump(DA_forecasts, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
ds = xr.load_dataset("_mars-bol-webmars-public-svc-blue-007-cc6e94d7ef7081c98337ea0fbc3ce669-iPWgUw.grib", engine="cfgrib")

ValueError: unrecognized engine cfgrib must be one of: ['scipy', 'store']

SyntaxError: invalid syntax (2874371567.py, line 1)